In [6]:

image_folder = '../../Images'
data_file_folder = '../../annotator_data'
annotator = 'Nikhil'
reviewer = 'Ankur'

In [7]:
import numpy as np
import pandas as pd
import cv2 as cv
import sys
import string
import os

In [8]:
kp_dataset_file = "KEYPOINTS_DATASET_" + annotator + '.csv'
status_file = 'STATUS_' + annotator + '.csv'
offered_file = 'offered_for_review_' + annotator + '.csv'

In [9]:
kp_dataset_path = os.path.join(data_file_folder, kp_dataset_file)
status_path = os.path.join(data_file_folder, status_file)
offered_path = os.path.join(data_file_folder, offered_file)

In [10]:
kp_dataset = pd.read_csv(kp_dataset_path, index_col = False)
status = pd.read_csv(status_path, index_col = False)


sent = status[status['sent_for_review'] == True ]
sent = sent['file_name'].values
all = status['file_name'].values
complete = status[status['success'] == True]
complete = complete['file_name'].values


for im in all:
     if(( im in sent) or (im not in complete)):
         kp_dataset = kp_dataset[kp_dataset['img_id'] != im]
     else:
         record = status['file_name'] == im
         status.loc[record,'sent_for_review'] = True
     
     



kp_dataset.to_csv(offered_path, index = False)

status.to_csv(status_path, index = False)